In [1]:
%%capture
import os
import sys

import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from utils.prompter import Prompter

In [2]:
# if torch.cuda.is_available():
#     device = "cuda"
device = "cuda"

In [3]:
load_8bit = False
base_model = 'decapoda-research/llama-7b-hf'
lora_weights = 'tloen/alpaca-lora-7b'
#lora_weights = "/workspace/cell-sales-chatbot/model-weights/alpaca-phone"
# The prompt template to use, will default to alpaca.
prompt_template = ""

In [4]:
tokenizer = LlamaTokenizer.from_pretrained(base_model)
if device == "cuda":
    alp_model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=load_8bit,
                    torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
#if can't find xxx.json error occurred, check the lora_weights variable, trying using full system path

if device == "cuda":
    alp_model = PeftModel.from_pretrained(alp_model, lora_weights, torch_dtype=torch.float16)

# unwind broken decapoda-research config
alp_model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
alp_model.config.bos_token_id = 1
alp_model.config.eos_token_id = 2

if not load_8bit:
    alp_model.half()  # seems to fix bugs for some users.

alp_model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    alp_model = torch.compile(alp_model)

In [6]:
prompter = Prompter(prompt_template)

def alpaca_inference(input_text, instructions, 
    temperature = 0.3, top_p = 0.75, top_k = 40, num_beams = 1, 
    max_new_tokens = 256, **kwargs):
    
    input_prompt = prompter.generate_prompt(instructions, input_text)
    generation_config = GenerationConfig(temperature=temperature, top_p=top_p,
        top_k=top_k, num_beams=num_beams, **kwargs)
    
    inputs = tokenizer(input_prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    with torch.no_grad():
        generation_output = alp_model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return prompter.get_response(output)

In [7]:
instruction = ""

input_text = "Hi, how are you?"

with torch.autocast("cuda"):
    output = alpaca_inference(input_text, instruction)

print(">>>>> Instruction:\n", input_text)
print(">>>>> Input:\n", input_text)
print("<<<<< Output:\n", output)

>>>>> Instruction:
 Hi, how are you?
>>>>> Input:
 Hi, how are you?
<<<<< Output:
 Hello, I am doing well. How are you?

### Instruction:


### Input:


In [8]:
query1 = "Can you list the battery life for the Apple iPhone SE, Xiaomi Redmi Note 9 Pro, and Huawei P30 Pro?"

In [9]:
instruction = "Extract phone model names\n\
output only the comma-separated model names"

input_text = query1

with torch.autocast("cuda"):
    output = alpaca_inference(input_text, instruction)

print(">>>>> Instruction:\n", instruction)
print(">>>>> Input:\n", input_text)
print("<<<<< Output:\n", output)

>>>>> Instruction:
 Extract phone model names
output only the comma-separated model names
>>>>> Input:
 Can you list the battery life for the Apple iPhone SE, Xiaomi Redmi Note 9 Pro, and Huawei P30 Pro?
<<<<< Output:
 Apple iPhone SE: 14 hours
Xiaomi Redmi Note 9 Pro: 5 hours
Huawei P30 Pro: 2 days


In [10]:
instruction = '''How many phones model names are there in the input?
'''

input_text = query1

with torch.autocast("cuda"):
    output = alpaca_inference(input_text, instruction)

print(">>>>> Instruction:\n", instruction)
print(">>>>> Input:\n", input_text)
print("<<<<< Output:\n", output)

>>>>> Instruction:
 How many phones model names are there in the input?

>>>>> Input:
 Can you list the battery life for the Apple iPhone SE, Xiaomi Redmi Note 9 Pro, and Huawei P30 Pro?
<<<<< Output:
 Apple iPhone SE: 1440 mAh
Xiaomi Redmi Note 9 Pro: 4000 mAh
Huawei P30 Pro: 4200 mAh


In [11]:
import json
import re
from tqdm import tqdm
import random
import pickle
from fuzzywuzzy import fuzz

# Load stuff
with open("phone_dataset.pkl", "rb") as f:
    pdb = pickle.load(f)
    
phonedb_data, name_map = pdb
name_list = list(name_map.keys())

def query_specs_list(short_name, debug=False, replace_new_line = True):
    spec_list = []
    for ln in name_map[short_name]:
        if debug:
            print(ln)
        if replace_new_line:
            spec = phonedb_data[ln][0].replace("\\n", "\n")
        else:
            spec = phonedb_data[ln][0]
        spec_list.append(spec)
    return spec_list

def query_key_text_list(short_name, debug=False, replace_new_line = True):
    key_text = []
    for ln in name_map[short_name]:
        if debug:
            print(ln)
        if replace_new_line:
            spec = phonedb_data[ln][0].replace("\\n", "\n")
        else:
            spec = phonedb_data[ln][0]
        key_text.append([ln, spec])
    return key_text

def fuzzy_score(sentence, word):
    return fuzz.partial_ratio(word.lower(), sentence.lower())

def fuzzy_scores(sentence, word_list):
    result = []
    for word in word_list:
        score = fuzz.partial_ratio(word.lower(), sentence.lower())
        result.append([word, score])
    return result    
def topk_lables(fuzzy_score_list, k = 5):
    fs_sort = sorted(fuzzy_score_list, key=lambda x: x[1], reverse=True)
    lbs = []
    if k < len(fs_sort):
        for i in range(k):
             lbs.append(fs_sort[i][0])
    else:
        for i in range(len(fs_sort)):
             lbs.append(fs_sort[i][0])
    return lbs
    

In [12]:
import torch
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")



def efficient_bart_cls_inference(text, long_label_list):
    narrowed_labels = topk_lables(fuzzy_scores(text, long_label_list)) #narrowed down to short name list
    result = classifier(text, narrowed_labels, multiclass=True)
    return result

#Example:
result = efficient_bart_cls_inference("Can you give me the weight for the Samsung Galaxy S21 Ultra?", 
                                    name_list)
pred_model_name = result["labels"][0]
print(pred_model_name)
    

Samsung Galaxy S21 Ultra


In [13]:
def name_query_mix_models_inference(sentence, model_name_list, print_process = False):
    ### Step 1: Alpaca extract name tokens
    instruction = "Ignore the input. Extract all phone model names from the input sentence. \
Append and prepend '%%%' symbols to each phone model name." 

    input_text = sentence
    
    if print_process:
        print("------------------------------------------------")
        print("Step 1: Alpaca extract name tokens")
        print("\n>>>>> Instruction:\n", instruction)
        print("\n>>>>> Input:\n", input_text)
        print("\nGenerating ......")
    
    with torch.autocast("cuda"):
        output = alpaca_inference(input_text, instruction, max_new_tokens = 128)
    
    output = output.split('###')[0].strip()
    output = output.strip()
    if print_process:
        print("\n<<<<< Output:\n", output)
    
    if print_process:
        print("\n------------------------------------------------")
        print("Using regex to tokenize:")
    matches = re.findall(r'%%([\w\s]+?)%%', output.replace("\n", ""))
    
    matches = list(set(matches))
    if print_process:
        print(matches)
    
    ### Step 2: iteratively call Bart classifier to get name keys for dict query
    ###         using the alpaca output as its input
    if print_process:
        print("------------------------------------------------")
        print("\nStep 2: Zero-shot BART classifier extract name keys\n")
    results = set()
    
    for token in matches:
        #Using fuzzy similarity scores to get top K candidate model names
        narrowed_labels = topk_lables(fuzzy_scores(token, model_name_list), k = 5) 
        
        cls_result = classifier(token, narrowed_labels, multiclass=True)
        pred = cls_result["labels"][0]
        results.add(pred)
        
        if print_process:
            print("Extracted Model Name: ", pred)
    
    if print_process:
        print("------------------------------------------------\n\n") 
            
            
    return results

In [14]:
query2 = "How does the Face ID feature on Apple iphone14 pro \
compare to the fingerprint sensor on the Huawei Mate40 Pro and Samsung Galaxy S10?"


names = name_query_mix_models_inference(query2, name_list, print_process=True)

------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 How does the Face ID feature on Apple iphone14 pro compare to the fingerprint sensor on the Huawei Mate40 Pro and Samsung Galaxy S10?

Generating ......

<<<<< Output:
 %%Apple iPhone14 Pro%%,%%Face ID%%,%%%%Compare%%,%%%%Fingerprint%%,%%%%Sensor%%,%%%%S10%%

------------------------------------------------
Using regex to tokenize:
['Fingerprint', 'Apple iPhone14 Pro', 'Face ID', 'S10', 'Sensor', 'Compare']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  Xiaomi Redmi Note 11T Pro
Extracted Model Name:  iPhone 14 Pro
Extracted Model Name:  Samsung Galaxy A9 Star Lite
Extracted Model Name:  Samsung Galaxy S10+
Extracted Model Name:  Xiaomi Mi 6 Mercury Silver
Extr

In [15]:
query5 = "What is the camera resolution for the Huawei P40?"


names = name_query_mix_models_inference(query5, name_list, print_process=True)

------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 What is the camera resolution for the Huawei P40?

Generating ......

<<<<< Output:
 %%Huawei P40%%Camera Resolution: 12MP+24MP+8MP+1080p@30fps+120fps@1080p+720p@30fps+2160p@30fps

------------------------------------------------
Using regex to tokenize:
['Huawei P40']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  Huawei P40
------------------------------------------------




In [16]:
query6 = "How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?"

names = name_query_mix_models_inference(query6, name_list, print_process=True)

------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?

Generating ......

<<<<< Output:
 %%Apple iPhone 12%%, %%Samsung Galaxy S21%%, %%Xiaomi Mi 11%%

------------------------------------------------
Using regex to tokenize:
['Xiaomi Mi 11', 'Samsung Galaxy S21', 'Apple iPhone 12']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  Xiaomi Mi 11
Extracted Model Name:  Samsung Galaxy S21
Extracted Model Name:  iPhone 12
------------------------------------------------




In [17]:
#Example to query phonedb database using mix models

query3 = "Can you list the battery life for the Apple iPhone 13, Xiaomi Redmi Note 9 Pro,\
and Huawei P30 Pro?"


key_names = name_query_mix_models_inference(query3, name_list, print_process=True)

print("\n------------------------------------------------")
print("Querying local DataBase ......")
for n in key_names:
    specs_texts = query_specs_list(n)
    print("++++++++++++++++++++++++++++++++++++++++++++++++")
    print(n, " related texts:")
    for text in specs_texts:
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        print(text[0:100], "\n") #only print the first 100 chars of a text
print("------------------------------------------------")

------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 Can you list the battery life for the Apple iPhone 13, Xiaomi Redmi Note 9 Pro,and Huawei P30 Pro?

Generating ......

<<<<< Output:
 %%Apple iPhone 13%%
%%Xiaomi Redmi Note 9 Pro%%
%%Huawei P30 Pro%%

------------------------------------------------
Using regex to tokenize:
['Apple iPhone 13', 'Xiaomi Redmi Note 9 Pro', 'Huawei P30 Pro']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  iPhone 13
Extracted Model Name:  Xiaomi Redmi Note 9 Pro
Extracted Model Name:  Huawei P30 Pro
------------------------------------------------



------------------------------------------------
Querying local DataBase ......
++++++++++++++++++++++++++++++++++++++++++++++++
Xiaom

In [18]:
from sentence_transformers import SentenceTransformer, util

def string_line_filter(string, filter_keywords, filter_out=True):
    lines = string.splitlines()
    filtered_lines = []
    for line in lines:
        append = True if filter_out else False
        for keyword in filter_keywords:
            if keyword in line:
                append = False if filter_out else True
        if append:
            filtered_lines.append(line)
    new_string = "\n".join(filtered_lines)
    
    return new_string

def extract_table_keys(text):
    lines = text.splitlines()
    keys = []
    for line in lines:
        key = line.split(":", maxsplit=1)[0]        
        keys.append(key.strip())
    return keys

def table_findall(text, keyword):
    pattern = r'.*\b(' + "Camera" + r')\b.*'
    matches = [line.strip() for line in text.split('\n') if re.match(pattern, line)]
    return matches


sentsim_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def sentence_similarity(sentsim_model, text1, text2):
    embedding_1= sentsim_model.encode(text1, convert_to_tensor=True)
    embedding_2 = sentsim_model.encode(text2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding_1, embedding_2)



In [19]:
# xxx = query_specs_list("iPhone 12")[0]


# keys = extract_table_keys(xxx)

# len(keys)

# table_findall(xxx, "camera")



In [20]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

def qa_inference(question, context, model_name="deepset/roberta-base-squad2"):
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {
        'question': question,
        'context': context    
    }
    return nlp(QA_input)

def relevant_table_text(question, text, topk = 3):
    lines = text.splitlines()
    relevant_text = ""
    scores = []
    for line in tqdm(lines):
        score = sentence_similarity(sentsim_transformer, question, line).item()
        scores.append([line, score])
    scores.sort(key=lambda x: x[1], reverse=True)
    for i in range(topk):
        relevant_text += scores[i][0] + "\n"
    return relevant_text



In [21]:
rel = relevant_table_text("How do the camera capabilities of the Apple iPhone 12,\
Samsung Galaxy S21, and Xiaomi Mi 11 compare?"
, query_specs_list("iPhone 12")[0])

print(rel)

100%|██████████| 158/158 [00:01<00:00, 92.79it/s]

 Brief Info: Compact iPhone model sharing most of its high-end features with the larger siblings in the 12 / 12 Pro model range, like A14 Bionic SoC, XDR AMOLED display, 12 MP wide and 12 MP ultra-wide rear cameras. Top 256 GB variant mainland for China, Hong Kong, Ma 
 Aux. Camera Extra Functions: HDR photo, Slow motion video, Burst mode, Touch focus, Panorama Photo, Face detection, Face tagging, Smile detection 
 Number of effective pixels: 12.2 MP camera 



In [22]:
# question = "How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?"

# key_names = name_query_mix_models_inference(question, name_list, print_process=True)




# print("\n------------------------------------------------")
# print("Querying local DataBase ......")
# for n in key_names:
#     specs_texts = query_specs_list(n)
#     print("++++++++++++++++++++++++++++++++++++++++++++++++")
#     print(n, " related texts:")
#     for text in specs_texts:
#         print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
#         relevant_text = relevant_table_text(question, text, topk=5)
        
#         print(qa_inference(question, context = relevant_text))
# print("------------------------------------------------")



In [23]:
question = "How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?"

key_names = name_query_mix_models_inference(question, name_list, print_process=True)




print("\n------------------------------------------------")
print("Querying local DataBase ......")

context_text = ""
for n in key_names:
    keys_texts = query_key_text_list(n)
    relevant_texts = []
    
    print("Model Name Family: ", n)
    
    # too many texts, crashed the alpaca model, added a simple filter
    topk = 3
    keys_list = []
    texts_list = []
    for key, text in keys_texts:
        keys_list.append(key)
        texts_list.append(text)
    cls_res = efficient_bart_cls_inference(question, keys_list)
    keys_texts_ranked = []
    ranked_keys = cls_res["labels"][0 : topk]
    for i in range(len(keys_list)):
        if keys_list[i] in ranked_keys:
            keys_texts_ranked.append([keys_list[i], texts_list[i]])
    
    
    for ln, text in keys_texts_ranked:
        relevant_text = "Model full name is '" + ln + "':\n"
        relevant_text += relevant_table_text(question, text, topk=3)
        relevant_text += "\n"
        #print(relevant_text)
        relevant_texts.append(relevant_text)
    sum_rele_text = ' '.join(relevant_texts)
    #print(sum_rele_text)
    
    # summarize
    instruction = '''Summarize the input passage which contains info about\
different specific models of a cellphone family
'''

    input_text = sum_rele_text
    with torch.autocast("cuda"):
        output = alpaca_inference(input_text, instruction)
    
    output = output.split('###')[0].strip()
    output = output.strip()
    context_text += output + "\n\n" 
    
        
print("------------------------------------------------")

print(".................................................")
print(context_text)




------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?

Generating ......

<<<<< Output:
 %%Apple iPhone 12%%, %%Samsung Galaxy S21%%, %%Xiaomi Mi 11%%

------------------------------------------------
Using regex to tokenize:
['Xiaomi Mi 11', 'Samsung Galaxy S21', 'Apple iPhone 12']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  Xiaomi Mi 11
Extracted Model Name:  Samsung Galaxy S21
Extracted Model Name:  iPhone 12
------------------------------------------------



------------------------------------------------
Querying local DataBase ......
Model Name Family:  iPhone 12


100%|██████████| 158/158 [00:01<00:00, 94.95it/s]


Model Name Family:  Xiaomi Mi 11


100%|██████████| 150/150 [00:01<00:00, 93.66it/s]


Model Name Family:  Samsung Galaxy S21


100%|██████████| 173/173 [00:01<00:00, 95.75it/s]


------------------------------------------------
.................................................
The iPhone 12 Mini 5G A2400 Dual SIM TD-LTE CN 64GB / A2401 is the most affordable 64 GB variant of iPhone 12 Mini for mainland China, Hong Kong, Macao featuring Super Retina XDR AMOLED display protected by Ceramic Shield, 12 + 12 MP wide-angle rear cameras and 12 MP TrueDepth front camera. It has HDR photo, Slow motion video, Burst mode, Touch focus, Panorama Photo, Face detection, Face tagging, Smile detection and 12.2 MP camera. The iPhone 12 5G A2404 Dual SIM TD-LTE CN 64GB / A2405 is the most affordable 64 GB variant of iPhone 12 for mainland China, Hong Kong and Macao featuring Super Retina XDR AMOLED display protected by Ceramic Shield, 12 + 12 MP wide-angle rear cameras and 12 MP TrueDepth front camera. It has HDR photo

Xiaomi Mi 11 Youth 5G Premium Edition Dual SIM TD-LTE CN 256GB M2101K9C is a Chinese variant of Mi11 Lite 5G smartphone with 256 GB UFS 2.2 ROM, 8 GiB LP-DDR4X RA

In [24]:
# instruction = '''Summarize the input passage which contains info about\
# different specific models of a cellphone family
# '''

# input_text = ltext

# with torch.autocast("cuda"):
#     output = alpaca_inference(input_text, instruction)

# print(">>>>> Instruction:\n", instruction)
# print("<<<<< Output:\n", output)

In [28]:
#Alpaca-base model fusion pipelines

question = "How do the camera capabilities of the Apple iPhone 12,\
Samsung Galaxy S21, and Xiaomi Mi 11 compare?"

key_names = name_query_mix_models_inference(question, name_list, print_process=True)


print("\n------------------------------------------------")
print("Querying local DataBase ......")

context_text = ""
for n in key_names:
    keys_texts = query_key_text_list(n)
    relevant_texts = []
    
    print("Model Name Family: ", n)
    
    # too many texts, crashed the alpaca model, added a simple filter
    topk = 3
    keys_list = []
    texts_list = []
    for key, text in keys_texts:
        keys_list.append(key)
        texts_list.append(text)
    cls_res = efficient_bart_cls_inference(question, keys_list)
    keys_texts_ranked = []
    ranked_keys = cls_res["labels"][0 : topk]
    for i in range(len(keys_list)):
        if keys_list[i] in ranked_keys:
            keys_texts_ranked.append([keys_list[i], texts_list[i]])
    
    
    for ln, text in keys_texts_ranked:
        relevant_text = "Model full name is '" + ln + "':\n"
        relevant_text += relevant_table_text(question, text, topk=3)
        relevant_text += "\n"
        #print(relevant_text)
        relevant_texts.append(relevant_text)
    sum_rele_text = ' '.join(relevant_texts)
    #print(sum_rele_text)
    
    # invoke Alpaca to summarize
    instruction = '''Summarize the input passage which contains info about\
different specific models of a cellphone family
'''
    input_text = sum_rele_text
    with torch.autocast("cuda"):
        output = alpaca_inference(input_text, instruction)
    
    output = output.split('###')[0].strip()
    output = output.strip()
    context_text += output + "\n\n" 
    
        
print("------------------------------------------------")

print(context_text)

print("------------------------------------------------")
print("................................................")




# Final inference to answer the question
instruction = "Answer the input question.\n\n\
You are given the following context information extracted from local database:\n\n"
instruction += context_text
input_text = question
with torch.autocast("cuda"):
    output = alpaca_inference(input_text, instruction)

output = output.split('###')[0].strip()
output = output.strip()

print("Back to the original question >>>>>> ")
print("------------------------------------------------")
print("Question:\n ", question)
print("\n\nAnswer:\n", output)


------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 How do the camera capabilities of the Apple iPhone 12,Samsung Galaxy S21, and Xiaomi Mi 11 compare?

Generating ......

<<<<< Output:
 %%Apple iPhone 12%%,%%Samsung Galaxy S21%%,%%Xiaomi Mi 11%%

------------------------------------------------
Using regex to tokenize:
['Xiaomi Mi 11', 'Samsung Galaxy S21', 'Apple iPhone 12']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  Xiaomi Mi 11
Extracted Model Name:  Samsung Galaxy S21
Extracted Model Name:  iPhone 12
------------------------------------------------



------------------------------------------------
Querying local DataBase ......
Model Name Family:  iPhone 12


100%|██████████| 158/158 [00:01<00:00, 96.31it/s]


Model Name Family:  Xiaomi Mi 11


100%|██████████| 150/150 [00:01<00:00, 94.48it/s]


Model Name Family:  Samsung Galaxy S21


100%|██████████| 173/173 [00:01<00:00, 94.69it/s]


------------------------------------------------
The iPhone 12 Mini 5G A2400 Dual SIM TD-LTE CN 128GB / A2401 is the most affordable variant of the iPhone 12 Mini for mainland China, Hong Kong and Macao. It features a Super Retina XDR AMOLED display protected by Ceramic Shield, 12 + 12 MP wide-angle rear cameras and 12 MP TrueDepth front camera. It also has HDR photo, Slow motion video, Burst mode, Touch focus, Panorama Photo, Face detection, Face tagging, Smile detection and 12.2 MP camera. The iPhone 12 Mini 5G A2400 Dual SIM TD-LTE CN 64GB / A2401 is the most affordable variant of the iPhone 12 Mini for mainland China, Hong Kong and Macao. It features a Super Retina XDR AMOLED display protected by Ceramic Shield, 12 + 12 MP wide-angle rear cameras and 12 MP TrueDepth front camera

Xiaomi Mi 11 Youth 5G Premium Edition Dual SIM TD-LTE CN 256GB M2101K9C is a Chinese variant of Mi11 Lite 5G smartphone with 256 GB UFS 2.2 ROM, 8 GiB LP-DDR4X RAM. Xiaomi Mi 11 Youth 5G Premium Edition Du

In [26]:
question = '''\
What is the maximum refresh rate of the iPhone 12 Pro \
Max display and how does it compare to the Samsung Galaxy Note20 Ultra?'''
key_names = name_query_mix_models_inference(question, name_list, print_process=True)


print("\n------------------------------------------------")
print("Querying local DataBase ......")

context_text = ""
for n in key_names:
    keys_texts = query_key_text_list(n)
    relevant_texts = []
    
    print("Model Name Family: ", n)
    
    # too many texts, crashed the alpaca model, added a simple filter
    topk = 3
    keys_list = []
    texts_list = []
    for key, text in keys_texts:
        keys_list.append(key)
        texts_list.append(text)
    cls_res = efficient_bart_cls_inference(question, keys_list)
    keys_texts_ranked = []
    ranked_keys = cls_res["labels"][0 : topk]
    for i in range(len(keys_list)):
        if keys_list[i] in ranked_keys:
            keys_texts_ranked.append([keys_list[i], texts_list[i]])
    
    
    for ln, text in keys_texts_ranked:
        relevant_text = "Model full name is '" + ln + "':\n"
        relevant_text += relevant_table_text(question, text, topk=3)
        relevant_text += "\n"
        #print(relevant_text)
        relevant_texts.append(relevant_text)
    sum_rele_text = ' '.join(relevant_texts)
    #print(sum_rele_text)
    
    # invoke Alpaca to summarize
    instruction = '''Summarize the input passage which contains info about\
different specific models of a cellphone family
'''
    input_text = sum_rele_text
    with torch.autocast("cuda"):
        output = alpaca_inference(input_text, instruction)
    
    output = output.split('###')[0].strip()
    output = output.strip()
    context_text += output + "\n\n" 
    
        
print("------------------------------------------------")

print(context_text)

print("------------------------------------------------")
print("................................................")




# Final inference to answer the question
instruction = "Answer the input question.\n\n\
You are given the following context information extracted from local database:\n\n"
instruction += context_text
input_text = question
with torch.autocast("cuda"):
    output = alpaca_inference(input_text, instruction)

output = output.split('###')[0].strip()
output = output.strip()

print("Back to the original question >>>>>> ")
print("------------------------------------------------")
print("Question:\n ", question)
print("\n\nAnswer:\n", output)



------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence. Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 What is the maximum refresh rate of the iPhone 12 Pro Max display and how does it compare to the Samsung Galaxy Note20 Ultra?

Generating ......

<<<<< Output:
 %%iPhone 12 Pro Max%% %%Samsung Galaxy Note20 Ultra%%

------------------------------------------------
Using regex to tokenize:
['iPhone 12 Pro Max', 'Samsung Galaxy Note20 Ultra']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  iPhone 12 Pro Max
Extracted Model Name:  Samsung Galaxy Note 20 Ultra
------------------------------------------------



------------------------------------------------
Querying local DataBase ......
Model Name Family:  iPhone 12 Pro Max


100%|██████████| 162/162 [00:01<00:00, 95.04it/s]


Model Name Family:  Samsung Galaxy Note 20 Ultra


100%|██████████| 167/167 [00:01<00:00, 94.14it/s]


------------------------------------------------
The iPhone 12 Pro Max 5G A2412 Dual SIM TD-LTE CN 512GB / A2413 has a 60Hz display refresh rate, a 120Hz touchscreen sampling rate, and a 60Hz display refresh rate. The iPhone 12 Pro Max 5G A2412 Dual SIM TD-LTE CN 256GB / A2413 has a 60Hz display refresh rate, a 120Hz touchscreen sampling rate, and a 60Hz display refresh rate. The iPhone 12 Pro Max 5G A2412 Dual SIM TD-LTE CN 128GB / A2413 has a 60Hz display refresh rate, a 120Hz touchscreen sampling rate, and a 60Hz display refresh rate.

The Samsung SM-N9860 Galaxy Note 20 Ultra 5G Dual SIM TD-LTE CN 512GB and the Samsung SM-N9860 Galaxy Note 20 Ultra 5G Dual SIM TD-LTE CN 256GB are two variants of the same phone, the Galaxy Note 20 Ultra 5G. Both variants have a 6.7-inch Super AMOLED display with a 120Hz refresh rate, a Snapdragon 865 chipset, and 512GB or 256GB of storage. The SM-N9860 Galaxy Note 20 Ultra 5G Dual SIM TD-LTE CN 512GB has a 512GB of storage, while the SM-N9860 Galaxy